In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem, RDLogger
from rdkit.Chem import AllChem, DataStructs, PandasTools

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score

import optuna
from optuna.samplers import TPESampler

import pickle

In [ ]:
def calc_max_tanimoto(train_id, test_id, df_init):
    morgan_train = df_init[df_init['Entry ID'].isin(train_id)][['Ligand ID','Ligand SMILES','Morgan']].drop_duplicates('Ligand ID').reset_index(drop = True)
    morgan_test = df_init[df_init['Entry ID'].isin(test_id)][['Ligand ID','Ligand SMILES','Morgan']].drop_duplicates('Ligand ID').reset_index(drop = True)

    train_fps = list(morgan_train['Morgan'])
    test_fps  = list(morgan_test['Morgan'])

    similarity_matrix = np.zeros((len(train_fps), len(test_fps)))

    for i, train_fp in enumerate(train_fps):
        sims = DataStructs.BulkTanimotoSimilarity(train_fp, test_fps)
        similarity_matrix[i, :] = np.round(sims, 4)  


    df_similarity = pd.DataFrame(similarity_matrix)
    df_similarity['max'] = df_similarity.max(axis = 1)
    max_tanimoto = df_similarity['max'].max()

    return max_tanimoto


def logreg_model_dev(train_id, df_train, df_test, folds, df_PB):
    
    def objective(trial):
        threshold = trial.suggest_float("threshold", 0.01, 0.99)

        result = []

        for i, fold_test in enumerate(folds):
            fold_train = [key for j, fold in enumerate(folds) if j != i for key in fold]

            df_train_CV = df_train[df_train['ID'].isin(fold_train)]
            df_test_CV = df_train[df_train['ID'].isin(fold_test)]

            X_train_CV = df_train_CV[['Smina', 'Gnina']]
            y_train_CV = df_train_CV['Eval']
            X_test_CV = df_test_CV[['Smina','Gnina']]
            y_test_CV = df_test_CV['Eval']

            scale = StandardScaler()
            X_train_scaled = scale.fit_transform(X_train_CV)
            X_test_scaled = scale.transform(X_test_CV)

            model = LogisticRegression()
            model.fit(X_train_scaled, y_train_CV)
            y_prob_test = model.predict_proba(X_test_scaled)[:,1]

            y_pred_test = (y_prob_test >= threshold).astype(int)
            result.append(balanced_accuracy_score(y_test_CV, y_pred_test))

        return np.mean(result)
    
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    sampler = TPESampler(seed=36)  
    study = optuna.create_study(direction="maximize", sampler=sampler)
    study.optimize(objective, n_trials=100)

    optimal_thres = round(study.best_params['threshold'],4)


    train_subset = df_train[df_train['ID'].isin(train_id)]

    X_train = train_subset[['Smina','Gnina']]
    y_train = train_subset['Eval']
    X_test = df_test[['Smina','Gnina']]
    y_test = df_test['Eval']

    scale = StandardScaler()
    X_train_scaled = scale.fit_transform(X_train)
    X_test_scaled = scale.transform(X_test)

    model = LogisticRegression()
    model.fit(X_train_scaled, y_train)

    y_prob_test = model.predict_proba(X_test_scaled)[:,1]
    y_pred_test = (y_prob_test >= optimal_thres).astype(int)
    acc_test = balanced_accuracy_score(y_test, y_pred_test)

    test_copy = df_test.copy()
    test_copy['Pred'] = y_prob_test
    test_ = test_copy.sort_values(['ID','RMSD'])
    test_new = test_.loc[test_.groupby('ID')['Pred'].nlargest(1,keep='first').index.get_level_values(1)]
    DP_test = test_new[test_new['Eval']==1].shape[0]/5201

    PB_new = df_PB[df_PB['Pose'].isin(test_new[test_new['Eval']==1]['Pose'])]
    DP_PB_test = PB_new.iloc[:,3:29].all(axis = 1).sum() / 5201

    acc_CV = []
    for i, fold_test in enumerate(folds):
        fold_train = [key for j, fold in enumerate(folds) if j != i for key in fold]

        df_train_CV = df_train[df_train['ID'].isin(fold_train)]
        df_test_CV = df_train[df_train['ID'].isin(fold_test)]

        X_train_CV = df_train_CV[['Smina', 'Gnina']]
        y_train_CV = df_train_CV['Eval']
        X_test_CV = df_test_CV[['Smina', 'Gnina']]
        y_test_CV = df_test_CV['Eval']

        scale = StandardScaler()
        X_train_scaled = scale.fit_transform(X_train_CV)
        X_test_scaled = scale.transform(X_test_CV)

        model = LogisticRegression()
        model.fit(X_train_scaled, y_train_CV)
        y_prob_test = model.predict_proba(X_test_scaled)[:, 1]
        y_pred_test = (y_prob_test >= optimal_thres).astype(int)

        acc = balanced_accuracy_score(y_test_CV, y_pred_test)
        acc_CV.append(acc)
    
    return round(DP_test,4), round(DP_PB_test,4), optimal_thres, round(acc_test,4), round(np.mean(acc_CV), 4), round(np.std(acc_CV), 4)

### Calculating the highest similarity between each training-(sub)set and all test set ligands

In [ ]:
df = pd.read_csv('df_init.csv')

RDLogger.DisableLog('rdApp.*')

PandasTools.AddMoleculeColumnToFrame(df, 'Ligand SMILES', 'Structure')
df['Morgan'] = df['Structure'].apply(lambda mol:AllChem.GetMorganFingerprintAsBitVect (mol, 2, 2048))

test_id = pd.read_csv(f'../NextTopDocker_ID/Training_Test/test.csv', header = None)
test_id = test_id.iloc[:,0]

for i in np.arange(10, 101, 10):
    train_id = pd.read_csv(f'../NextTopDocker_ID/Training_Test/train_{i}.csv', header = None)
    train_id = train_id.iloc[:,0]

    max_tanimoto = calc_max_tanimoto(train_id, test_id, df)

    print(f"Training set LogReg ({i}%): {max_tanimoto}")

### Develop 10 LogReg (x%) models and assess their docking power and classification power

In [ ]:
train = pd.read_csv('df_train.csv')
test = pd.read_csv('df_test.csv')

PB = pd.read_csv('Test_PoseBuster.csv')
PB['Pose'] = PB['file'].apply(lambda x: str(x).split('/')[-1].split('.')[0])


for i in np.arange(10, 101, 10):
    train_id = pd.read_csv(f'../NextTopDocker_ID/Training_Test/train_{i}.csv', header = None)
    train_id = train_id.iloc[:,0]

    with open(f'../NextTopDocker_ID/5CV_ID/5CV_train_{i}.pkl', 'rb') as f:
        folds = pickle.load(f)

    result = logreg_model_dev(train_id, train, test, folds, PB)

    print(f"LogReg ({i}%)")
    print(f"Docking power: DP test ({result[0]}), DP test with PB ({result[1]})")
    print(f"Classification power: Optimal threshold ({result[2]}), BA on test ({result[3]}), BA on 5CV ({result[4]} +- {result[5]}))")
    print("----------")


### Analysis docking power of SOTA end-to-end ML docking tools

In [ ]:
#DeepDock
print("DeepDock results:")
df_deepdock_RMSD = pd.read_csv("../Result_SOTA/DeepDock_final_RMSD.csv")
print(f"DP: {round(df_deepdock_RMSD[df_deepdock_RMSD['RMSD']<=2].shape[0]/5201,4)}")

df_deepdock_PB = pd.read_csv("../Result_SOTA/DeepDock_PoseBuster.csv")
print(f"DP with PB: {round(df_deepdock_PB.iloc[:,3:29].all(axis = 1).sum()/5201,4)}")

In [ ]:
#Interformer (Energy Score)
print("Interformer (Energy Score) results:")

df_int_energy_RMSD = pd.read_csv("../Result_SOTA/Interformer_energy_final_RMSD.csv")
df_int_energy_RMSD = df_int_energy_RMSD.sort_values(['Target','rmsd'])
df_int_energy_RMSD = df_int_energy_RMSD.loc[df_int_energy_RMSD.groupby('Target')['energy'].nsmallest(1, keep='first').index.get_level_values(1)]
print(f"DP: {round(df_int_energy_RMSD[df_int_energy_RMSD['rmsd']<=2].shape[0]/5201,4)}")

df_int_energy_PB = pd.read_csv("../Result_SOTA/Interformer_energy_PoseBuster.csv")
df_int_energy_PB['Pose'] = df_int_energy_PB['file'].apply(lambda x: str(x).split('/')[-1].split('_')[0])
df_int_energy_PB = df_int_energy_PB[df_int_energy_PB['Pose'].isin(df_int_energy_RMSD[df_int_energy_RMSD['rmsd']<=2]['Target'])]
print(f"DP with PB: {round(df_int_energy_PB.iloc[:,3:29].all(axis = 1).sum()/5201,4)}")

In [ ]:
#Interformer (Pose Score)
print("Interformer (Pose Score) results:")

df_int_posescore_RMSD = pd.read_csv("../Result_SOTA/Interformer_posescore_final_RMSD.csv")
df_int_posescore_RMSD = df_int_posescore_RMSD.sort_values(['Target','rmsd'])
df_int_posescore_RMSD = df_int_posescore_RMSD.loc[df_int_posescore_RMSD.groupby('Target')['pred_pose'].nlargest(1, keep='first').index.get_level_values(1)]
print(f"DP: {round(df_int_posescore_RMSD[df_int_posescore_RMSD['rmsd']<=2].shape[0]/5201,4)}")

df_int_posescore_PB = pd.read_csv("../Result_SOTA/Interformer_posescore_PoseBuster.csv")
df_int_posescore_PB['Pose'] = df_int_posescore_PB['file'].apply(lambda x: str(x).split('/')[-1].split('_')[0])
df_int_posescore_PB = df_int_posescore_PB[df_int_posescore_PB['Pose'].isin(df_int_posescore_RMSD[df_int_posescore_RMSD['rmsd']<=2]['Target'])]
print(f"DP with PB: {round(df_int_posescore_PB.iloc[:,3:29].all(axis = 1).sum()/5201,4)}")

In [ ]:
#SurfDock
print("SurfDock results:")

df_surfdock_RMSD = pd.read_csv("../Result_SOTA/SurfDock_final_RMSD.csv", header = None)
df_surfdock_RMSD.columns = ['ID','RMSD']
print(f"DP: {round(df_surfdock_RMSD[df_surfdock_RMSD['RMSD']<=2].shape[0]/5201, 4)}")

df_surfdock_PB = pd.read_csv("../Result_SOTA/SurfDock_PoseBuster.csv")
df_surfdock_PB['Pose'] = df_surfdock_PB['molecule'].apply(lambda x: str(x).split('_')[0])
df_surfdock_PB = df_surfdock_PB[df_surfdock_PB['Pose'].isin(df_surfdock_RMSD[df_surfdock_RMSD['RMSD']<=2]['ID'])]
print(f"DP with PB: {round(df_surfdock_PB.iloc[:,3:29].all(axis = 1).sum()/5201, 4)}")

In [ ]:
#Uni-Mol Docking v.2
print("Uni-Mol Docking v.2 results:")

df_UM2_RMSD = pd.read_csv("../Result_SOTA/UM2_final_RMSD.csv", header = None)
df_UM2_RMSD.columns = ['ID','RMSD']
print(f"DP: {round(df_UM2_RMSD[df_UM2_RMSD['RMSD']<=2].shape[0]/5201, 4)}")

df_UM2_PB = pd.read_csv("../Result_SOTA/UM2_PoseBuster.csv")
df_UM2_PB['Pose'] = df_UM2_PB['file'].apply(lambda x: str(x).split('/')[1])
df_UM2_PB = df_UM2_PB[df_UM2_PB['Pose'].isin(df_UM2_RMSD[df_UM2_RMSD['RMSD']<=2]['ID'])]
print(f"DP with PB: {round(df_UM2_PB.iloc[:,3:29].all(axis = 1).sum()/5201, 4)}")